In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'
from ML import *
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
print("GPU is","avaliable" if tf.config.experimental.list_physical_devices("GPU") else "not available")

GPU is not available


In [2]:
'''
We first import the necessary libraries and LOAD all the data for training and testing. Such data corresponds to the original samples from DeltaPhish (either benign or malicious) as well as the 100 phishing samples (not used in the training data) that we use as basis to create our adversarial examples.

We then develop a given leaner (i.e., a phishing detector) by using the train data and a Convolutional Neural Network algorithm, and test it on the test data, computing the FPR and TPR shown in Table 3; we also compute the TPR on the subset of 100 samples used for our adversarial evaluations, which correspond to the results in Figure 4b (for the LR algorithm).

We then proceed to assess the robustness of the detector against ALL the 12 attacks considered in our paper, all of which use the 100 samples as basis. For WA and WA+, we already provide the preprocessed version. For PA, we use the samples created during the preprocessing phase. For MA, we create the samples at runtime.
'''

'\nWe first import the necessary libraries and LOAD all the data for training and testing. Such data corresponds to the original samples from DeltaPhish (either benign or malicious) as well as the 100 phishing samples (not used in the training data) that we use as basis to create our adversarial examples.\n\nWe then develop a given leaner (i.e., a phishing detector) by using the train data and a Convolutional Neural Network algorithm, and test it on the test data, computing the FPR and TPR shown in Table 3; we also compute the TPR on the subset of 100 samples used for our adversarial evaluations, which correspond to the results in Figure 4b (for the LR algorithm).\n\nWe then proceed to assess the robustness of the detector against ALL the 12 attacks considered in our paper, all of which use the 100 samples as basis. For WA and WA+, we already provide the preprocessed version. For PA, we use the samples created during the preprocessing phase. For MA, we create the samples at runtime.\n'

In [3]:
# load data and split test and train data
json_file='../data_folder/deltaphish/preprocessed/normal/deltaphish_total_feature_modified.json'
data_delta=data_load(json_file)
# remove evasion dataset, then build new PWDs using remain data
phish_sub_test_x=joblib.load('../data_folder/deltaphish/preprocessed/phish_sub_test_x_100.pkl')
#print('phish_test_x index',phish_sub_test_x.index)#100
#print('phish_sub_test_x shape',phish_sub_test_x.shape)
data_delta_change=data_delta.copy()
#get the 100 phish samples and add them to the test dataset
phish_sub_test_later=data_delta_change.loc[phish_sub_test_x.index].copy()
data_delta_change=data_delta_change.drop(phish_sub_test_x.index)# del those 100 samples

train_random,test_random=train_test_split(data_delta_change,train_size=3200,test_size=700,shuffle=True,random_state=501)
train_x0=train_random.iloc[:,0:58]
train_y0=train_random.iloc[:,58]

#test_later=pd.concat(test_random,phish_sub_test_later)
frames =[test_random,phish_sub_test_later]
test_later=pd.concat(frames)
test_x0=test_later.iloc[:,0:58] 
test_y0=test_later.iloc[:,58] 

#use only html features
train_html_x0=train_random.iloc[:,27:49]
test_html_x0=test_later.iloc[:,27:49]

#only url features
train_url_x0=pd.concat([train_random.iloc[:,0:27],train_random.iloc[:,49:58]],axis=1)
test_url_x0=pd.concat([test_later.iloc[:,0:27],test_later.iloc[:,49:58]],axis=1)
train_y1=train_url_y0=train_html_y0=train_y0
test_y1=test_url_y0=test_html_y0=test_y0

#benign data 
benign_index=test_y0[test_y0==0].index.tolist()#
phish_index=test_y0[test_y0==1].index.tolist()#

#generate benign_test_x_y
benign_test_x=test_x0.loc[benign_index,:]
benign_test_y= test_y0.loc[benign_index]# 
benign_test_url_x=pd.concat([benign_test_x.iloc[:,0:27],benign_test_x.iloc[:,49:58]],axis=1)
benign_test_html_x=benign_test_x.iloc[:,27:49]
#100 phishing data already selected from test data to calculate the baseline recall
phish_sub_test_y=test_y0[phish_sub_test_x.index]#100 phishing samples
phish_sub_test_html_x=phish_sub_test_x.iloc[:,27:49]
phish_sub_test_url_x=pd.concat([phish_sub_test_x.iloc[:,0:27],phish_sub_test_x.iloc[:,49:58]],axis=1)

# data format for cnn
one_hot_train_y0=tf.one_hot(indices=train_y0.array,depth=2,on_value=1.0,off_value=0.0)
one_hot_test_y0=tf.one_hot(indices=test_y0.array,depth=2,on_value=1.0,off_value=0.0)
test_x_re0=np.asarray(test_x0).reshape(test_x0.shape[0],test_x0.shape[1],1)
train_x_re0=np.asarray(train_x0).reshape(train_x0.shape[0],train_x0.shape[1],1)

train_html_x_re0=np.asarray(train_html_x0).reshape(train_html_x0.shape[0],train_html_x0.shape[1],1)
test_html_x_re0=np.asarray(test_html_x0).reshape(test_html_x0.shape[0],test_html_x0.shape[1],1)
train_url_x_re0=np.asarray(train_url_x0).reshape(train_url_x0.shape[0],train_url_x0.shape[1],1)
test_url_x_re0=np.asarray(test_url_x0).reshape(test_url_x0.shape[0],test_url_x0.shape[1],1)

benign_test_x_re=np.asarray(benign_test_x).reshape(benign_test_x.shape[0],benign_test_x.shape[1],1)
benign_test_html_x_re=np.asarray(benign_test_html_x).reshape(benign_test_html_x.shape[0],benign_test_html_x.shape[1],1)
benign_test_url_x_re=np.asarray(benign_test_url_x).reshape(benign_test_url_x.shape[0],benign_test_url_x.shape[1],1)


phish_sub_test_x_re=np.asarray(phish_sub_test_x).reshape(phish_sub_test_x.shape[0],phish_sub_test_x.shape[1],1)
phish_sub_test_html_x_re=np.asarray(phish_sub_test_html_x).reshape(phish_sub_test_html_x.shape[0],phish_sub_test_html_x.shape[1],1)
phish_sub_test_url_x_re=np.asarray(phish_sub_test_url_x).reshape(phish_sub_test_url_x.shape[0],phish_sub_test_url_x.shape[1],1)



# training

In [4]:
model_cnn_combined=create_model_cnn()
model_cnn_url=create_model_url()
model_cnn_html=create_model_html()

print('***************fit cnn_combined model:')
model_cnn_combined.fit(train_x_re0,one_hot_train_y0,batch_size=32,epochs=200, verbose=2)
print('***************fit cnn_url model:')
model_cnn_url.fit(train_url_x_re0,one_hot_train_y0,batch_size=32,epochs=200, verbose=2)
print('***************fit cnn_html model:')
model_cnn_html.fit(train_html_x_re0,one_hot_train_y0,batch_size=32,epochs=200, verbose=2)

***************fit cnn_combined model:
Epoch 1/200
100/100 - 1s - loss: 0.3830 - categorical_accuracy: 0.8675 - 1s/epoch - 12ms/step
Epoch 2/200
100/100 - 0s - loss: 0.3039 - categorical_accuracy: 0.8816 - 396ms/epoch - 4ms/step
Epoch 3/200
100/100 - 0s - loss: 0.2648 - categorical_accuracy: 0.8950 - 365ms/epoch - 4ms/step
Epoch 4/200
100/100 - 1s - loss: 0.2220 - categorical_accuracy: 0.9194 - 539ms/epoch - 5ms/step
Epoch 5/200
100/100 - 0s - loss: 0.1926 - categorical_accuracy: 0.9294 - 428ms/epoch - 4ms/step
Epoch 6/200
100/100 - 0s - loss: 0.1686 - categorical_accuracy: 0.9406 - 353ms/epoch - 4ms/step
Epoch 7/200
100/100 - 0s - loss: 0.1595 - categorical_accuracy: 0.9441 - 371ms/epoch - 4ms/step
Epoch 8/200
100/100 - 0s - loss: 0.1595 - categorical_accuracy: 0.9434 - 370ms/epoch - 4ms/step
Epoch 9/200
100/100 - 0s - loss: 0.1502 - categorical_accuracy: 0.9466 - 380ms/epoch - 4ms/step
Epoch 10/200
100/100 - 0s - loss: 0.1444 - categorical_accuracy: 0.9503 - 372ms/epoch - 4ms/step
Ep

***************fit cnn_url model:
Epoch 1/200
100/100 - 2s - loss: 0.4181 - categorical_accuracy: 0.8603 - 2s/epoch - 16ms/step
Epoch 2/200
100/100 - 1s - loss: 0.3518 - categorical_accuracy: 0.8644 - 553ms/epoch - 6ms/step
Epoch 3/200
100/100 - 1s - loss: 0.3386 - categorical_accuracy: 0.8744 - 578ms/epoch - 6ms/step
Epoch 4/200
100/100 - 1s - loss: 0.3263 - categorical_accuracy: 0.8744 - 551ms/epoch - 6ms/step
Epoch 5/200
100/100 - 1s - loss: 0.3278 - categorical_accuracy: 0.8753 - 556ms/epoch - 6ms/step
Epoch 6/200
100/100 - 1s - loss: 0.3195 - categorical_accuracy: 0.8794 - 562ms/epoch - 6ms/step
Epoch 7/200
100/100 - 1s - loss: 0.3235 - categorical_accuracy: 0.8809 - 588ms/epoch - 6ms/step
Epoch 8/200
100/100 - 1s - loss: 0.3182 - categorical_accuracy: 0.8763 - 590ms/epoch - 6ms/step
Epoch 9/200
100/100 - 1s - loss: 0.3078 - categorical_accuracy: 0.8822 - 554ms/epoch - 6ms/step
Epoch 10/200
100/100 - 1s - loss: 0.3044 - categorical_accuracy: 0.8872 - 632ms/epoch - 6ms/step
Epoch 1

***************fit cnn_html model:
Epoch 1/200
100/100 - 1s - loss: 0.3913 - categorical_accuracy: 0.8522 - 744ms/epoch - 7ms/step
Epoch 2/200
100/100 - 0s - loss: 0.2863 - categorical_accuracy: 0.8869 - 197ms/epoch - 2ms/step
Epoch 3/200
100/100 - 0s - loss: 0.2343 - categorical_accuracy: 0.9231 - 193ms/epoch - 2ms/step
Epoch 4/200
100/100 - 0s - loss: 0.2153 - categorical_accuracy: 0.9306 - 198ms/epoch - 2ms/step
Epoch 5/200
100/100 - 0s - loss: 0.2056 - categorical_accuracy: 0.9391 - 191ms/epoch - 2ms/step
Epoch 6/200
100/100 - 0s - loss: 0.1944 - categorical_accuracy: 0.9403 - 189ms/epoch - 2ms/step
Epoch 7/200
100/100 - 0s - loss: 0.1917 - categorical_accuracy: 0.9394 - 194ms/epoch - 2ms/step
Epoch 8/200
100/100 - 0s - loss: 0.1887 - categorical_accuracy: 0.9438 - 183ms/epoch - 2ms/step
Epoch 9/200
100/100 - 0s - loss: 0.1889 - categorical_accuracy: 0.9413 - 184ms/epoch - 2ms/step
Epoch 10/200
100/100 - 0s - loss: 0.1786 - categorical_accuracy: 0.9438 - 187ms/epoch - 2ms/step
Epoc

# baseline performance- refer to Table 3, top right cell

In [5]:

print('cnn_recall_com,whole:')    
cnn_recall_com=get_base_recall_cnn(model_cnn_combined,test_x_re0,test_y0)
print('cnn_recall_url,whole:')  
cnn_recall_url=get_base_recall_cnn(model_cnn_url,test_url_x_re0,test_url_y0)
print('cnn_recall_html,whole:')  
cnn_recall_html=get_base_recall_cnn(model_cnn_html,test_html_x_re0,test_html_y0)

print('cnn_fpr_com,whole:')    
cnn_fpr_com=get_base_fpr_cnn(model_cnn_combined,test_x_re0,test_y0)
print('cnn_fpr_url,whole:')  
cnn_fpr_url=get_base_fpr_cnn(model_cnn_url,test_url_x_re0,test_url_y0)
print('cnn_fpr_html,whole:')  
cnn_fpr_html=get_base_fpr_cnn(model_cnn_html,test_html_x_re0,test_html_y0)


cnn_recall_com,whole:
cnn_recall 0.92
cnn_recall_url,whole:
cnn_recall 0.53
cnn_recall_html,whole:
cnn_recall 0.81
cnn_fpr_com,whole:
fpr 0.003
cnn_fpr_url,whole:
fpr 0.043
cnn_fpr_html,whole:
fpr 0.013


# no-atk,refer to Fig 4b, the green bars in the leftmost subplot

In [6]:
print('cnn_no-atk_recall_combine:')
base_sub_cnn_recall_com=get_base_recall_cnn(model_cnn_combined,phish_sub_test_x_re,phish_sub_test_y)
print('cnn_no-atk_recall_url:')
base_sub_cnn_recall_url=get_base_recall_cnn(model_cnn_url,phish_sub_test_url_x_re,phish_sub_test_y)
print('cnn_no-atk_recall_html:')
base_sub_cnn_recall_html=get_base_recall_cnn(model_cnn_html,phish_sub_test_html_x_re,phish_sub_test_y)
 


cnn_no-atk_recall_combine:
cnn_recall 0.92
cnn_no-atk_recall_url:
cnn_recall 0.63
cnn_no-atk_recall_html:
cnn_recall 0.82


# wa_u attack- refer to Fig 4b, the orange bars in the leftmost subplot

In [7]:
ad_url_problem_data=data_load("../data_folder/deltaphish/preprocessed/wa/wa_u/phish_sub_problem_space_url_shorten100.json")
ad_url_problem_test_x=ad_url_problem_data.iloc[:,0:58]
ad_url_problem_test_y=ad_url_problem_data.iloc[:,58]

ad_url_problem_test_url_x=pd.concat([ad_url_problem_data.iloc[:,0:27],ad_url_problem_data.iloc[:,49:58]],axis=1)
ad_url_problem_test_html_x=ad_url_problem_data.iloc[:,27:49]

ad_problem_test_x_re=np.asarray(ad_url_problem_test_x).reshape(ad_url_problem_test_x.shape[0],ad_url_problem_test_x.shape[1],1)
ad_problem_test_url_x_re=np.asarray(ad_url_problem_test_url_x).reshape(ad_url_problem_test_url_x.shape[0],ad_url_problem_test_url_x.shape[1],1)
ad_problem_test_html_x_re=np.asarray(ad_url_problem_test_html_x).reshape(ad_url_problem_test_html_x.shape[0],ad_url_problem_test_html_x.shape[1],1)

#...............
print('cnn_recall_combined_wa_u:')
base_sub_cnn_recall_com=get_base_recall_cnn(model_cnn_combined,ad_problem_test_x_re,phish_sub_test_y)
print('cnn_recall_url_wa_u:')
base_sub_cnn_recall_url=get_base_recall_cnn(model_cnn_url,ad_problem_test_url_x_re,phish_sub_test_y)
print('cnn_recall_html_wa_u:')
base_sub_cnn_recall_html=get_base_recall_cnn(model_cnn_html,ad_problem_test_html_x_re,phish_sub_test_y)



cnn_recall_combined_wa_u:
cnn_recall 0.91
cnn_recall_url_wa_u:
recall 1
cnn_recall_html_wa_u:
cnn_recall 0.81


# wa_r attack- refer to Fig 4b, the firebrick bars in the leftmost subplot

In [8]:
ad_problem_data=data_load("../data_folder/deltaphish/preprocessed/wa/wa_r/phish_sub_problem_space_html_add50internallinks.json")#wa_u,wa_c
ad_problem_test_x=ad_problem_data.iloc[:,0:58]
ad_problem_test_y=ad_problem_data.iloc[:,58]

ad_problem_test_url_x=pd.concat([ad_problem_data.iloc[:,0:27],ad_problem_data.iloc[:,49:58]],axis=1)
ad_problem_test_html_x=ad_problem_data.iloc[:,27:49]

ad_problem_test_x_re=np.asarray(ad_problem_test_x).reshape(ad_problem_test_x.shape[0],ad_problem_test_x.shape[1],1)
ad_problem_test_url_x_re=np.asarray(ad_problem_test_url_x).reshape(ad_problem_test_url_x.shape[0],ad_problem_test_url_x.shape[1],1)
ad_problem_test_html_x_re=np.asarray(ad_problem_test_html_x).reshape(ad_problem_test_html_x.shape[0],ad_problem_test_html_x.shape[1],1)

print('cnn_recall_combined_wa_r:')
base_sub_cnn_recall_com=get_base_recall_cnn(model_cnn_combined,ad_problem_test_x_re,phish_sub_test_y)
print('cnn_recall_url_wa_r:')
base_sub_cnn_recall_url=get_base_recall_cnn(model_cnn_url,ad_problem_test_url_x_re,phish_sub_test_y)
print('cnn_recall_html_wa_r:')
base_sub_cnn_recall_html=get_base_recall_cnn(model_cnn_html,ad_problem_test_html_x_re,phish_sub_test_y)



cnn_recall_combined_wa_r:
cnn_recall 0.95
cnn_recall_url_wa_r:
cnn_recall 0.63
cnn_recall_html_wa_r:
cnn_recall 0.38


# wa_c attack- refer to Fig 4b, the darkred bars in the leftmost subplot

In [9]:
ad_problem_data=data_load("../data_folder/deltaphish/preprocessed/wa/wa_c/phish_sub_problem_space_html_add50internallinks_combined_shorten_url.json")
ad_problem_test_x=ad_problem_data.iloc[:,0:58]
ad_problem_test_y=ad_problem_data.iloc[:,58]
 
ad_problem_test_url_x=pd.concat([ad_problem_data.iloc[:,0:27],ad_problem_data.iloc[:,49:58]],axis=1)
ad_problem_test_html_x=ad_problem_data.iloc[:,27:49]

ad_problem_test_x_re=np.asarray(ad_problem_test_x).reshape(ad_problem_test_x.shape[0],ad_problem_test_x.shape[1],1)
ad_problem_test_url_x_re=np.asarray(ad_problem_test_url_x).reshape(ad_problem_test_url_x.shape[0],ad_problem_test_url_x.shape[1],1)
ad_problem_test_html_x_re=np.asarray(ad_problem_test_html_x).reshape(ad_problem_test_html_x.shape[0],ad_problem_test_html_x.shape[1],1)

print('cnn_recall_combined_wa_c:')
base_sub_cnn_recall_com=get_base_recall_cnn(model_cnn_combined,ad_problem_test_x_re,phish_sub_test_y)
print('cnn_recall_url_wa_c:')
base_sub_cnn_recall_url=get_base_recall_cnn(model_cnn_url,ad_problem_test_url_x_re,phish_sub_test_y)
print('cnn_recall_html_wa_c:')
base_sub_cnn_recall_html=get_base_recall_cnn(model_cnn_html,ad_problem_test_html_x_re,phish_sub_test_y)




cnn_recall_combined_wa_c:
cnn_recall 0.92
cnn_recall_url_wa_c:
recall 1
cnn_recall_html_wa_c:
cnn_recall 0.37


# wa+_u attack- refer to Fig 5b, bottom row,  green bars of the leftmost group of bars

In [10]:
ad_problem_data=data_load("../data_folder/deltaphish/preprocessed/wa+/wa+_u/phish_sub_html_change_full_knowledge_change_again_later_only_url_shorten.json")
ad_problem_test_x=ad_problem_data.iloc[:,0:58]
ad_problem_test_y=ad_problem_data.iloc[:,58]

ad_problem_test_url_x=pd.concat([ad_problem_data.iloc[:,0:27],ad_problem_data.iloc[:,49:58]],axis=1)
ad_problem_test_html_x=ad_problem_data.iloc[:,27:49]

ad_problem_test_x_re=np.asarray(ad_problem_test_x).reshape(ad_problem_test_x.shape[0],ad_problem_test_x.shape[1],1)
ad_problem_test_url_x_re=np.asarray(ad_problem_test_url_x).reshape(ad_problem_test_url_x.shape[0],ad_problem_test_url_x.shape[1],1)
ad_problem_test_html_x_re=np.asarray(ad_problem_test_html_x).reshape(ad_problem_test_html_x.shape[0],ad_problem_test_html_x.shape[1],1)

print('cnn_recall_combined_wa+_u:')
base_sub_cnn_recall_com=get_base_recall_cnn(model_cnn_combined,ad_problem_test_x_re,phish_sub_test_y)
print('cnn_recall_url_wa+_u:')
base_sub_cnn_recall_url=get_base_recall_cnn(model_cnn_url,ad_problem_test_url_x_re,phish_sub_test_y)
print('cnn_recall_html_wa+_u:')
base_sub_cnn_recall_html=get_base_recall_cnn(model_cnn_html,ad_problem_test_html_x_re,phish_sub_test_y)



cnn_recall_combined_wa+_u:
cnn_recall 0.94
cnn_recall_url_wa+_u:
recall 1
cnn_recall_html_wa+_u:
cnn_recall 0.81


# wa+_r attack- refer to Fig 5b, bottom row,  blue bars of the leftmost group of bars

In [11]:

ad_problem_data=data_load("../data_folder/deltaphish/preprocessed/wa+/wa+_r/phish_sub_problem_space__full_onlyhtml_316.json")#wa+_u,wa+_c

ad_problem_test_x=ad_problem_data.iloc[:,0:58]
ad_problem_test_y=ad_problem_data.iloc[:,58]

ad_problem_test_url_x=pd.concat([ad_problem_data.iloc[:,0:27],ad_problem_data.iloc[:,49:58]],axis=1)
ad_problem_test_html_x=ad_problem_data.iloc[:,27:49]

ad_problem_test_x_re=np.asarray(ad_problem_test_x).reshape(ad_problem_test_x.shape[0],ad_problem_test_x.shape[1],1)
ad_problem_test_url_x_re=np.asarray(ad_problem_test_url_x).reshape(ad_problem_test_url_x.shape[0],ad_problem_test_url_x.shape[1],1)
ad_problem_test_html_x_re=np.asarray(ad_problem_test_html_x).reshape(ad_problem_test_html_x.shape[0],ad_problem_test_html_x.shape[1],1)

print('cnn_recall_combined_wa+_r:')
base_sub_cnn_recall_com=get_base_recall_cnn(model_cnn_combined,ad_problem_test_x_re,phish_sub_test_y)
print('cnn_recall_url_wa+_r:')
base_sub_cnn_recall_url=get_base_recall_cnn(model_cnn_url,ad_problem_test_url_x_re,phish_sub_test_y)
print('cnn_recall_html_wa+_r:')
base_sub_cnn_recall_html=get_base_recall_cnn(model_cnn_html,ad_problem_test_html_x_re,phish_sub_test_y)


cnn_recall_combined_wa+_r:
cnn_recall 0.86
cnn_recall_url_wa+_r:
cnn_recall 0.63
cnn_recall_html_wa+_r:
cnn_recall 0.83


# wa+_c attack- refer to Fig 5b, bottom row, red bars of the leftmost group of bars

In [12]:
ad_problem_data=data_load("../data_folder/deltaphish/preprocessed/wa+/wa+_c/phish_sub_problem_space_mix_url_html_316.json")
ad_problem_test_x=ad_problem_data.iloc[:,0:58]
ad_problem_test_y=ad_problem_data.iloc[:,58]

ad_problem_test_url_x=pd.concat([ad_problem_data.iloc[:,0:27],ad_problem_data.iloc[:,49:58]],axis=1)
ad_problem_test_html_x=ad_problem_data.iloc[:,27:49]

ad_problem_test_x_re=np.asarray(ad_problem_test_x).reshape(ad_problem_test_x.shape[0],ad_problem_test_x.shape[1],1)
ad_problem_test_url_x_re=np.asarray(ad_problem_test_url_x).reshape(ad_problem_test_url_x.shape[0],ad_problem_test_url_x.shape[1],1)
ad_problem_test_html_x_re=np.asarray(ad_problem_test_html_x).reshape(ad_problem_test_html_x.shape[0],ad_problem_test_html_x.shape[1],1)

print('cnn_recall_combined_wa+_c:')
base_sub_cnn_recall_com=get_base_recall_cnn(model_cnn_combined,ad_problem_test_x_re,phish_sub_test_y)
print('cnn_recall_url_wa+_c:')
base_sub_cnn_recall_url=get_base_recall_cnn(model_cnn_url,ad_problem_test_url_x_re,phish_sub_test_y)
print('cnn_recall_html_wa+_c:')
base_sub_cnn_recall_html=get_base_recall_cnn(model_cnn_html,ad_problem_test_html_x_re,phish_sub_test_y)


cnn_recall_combined_wa+_c:
cnn_recall 0.93
cnn_recall_url_wa+_c:
recall 1
cnn_recall_html_wa+_c:
cnn_recall 0.86


# pa_u attack- refer to Fig 5b, bottom row, green bars  in middle group of bars

In [13]:

phish_sub_test_x_ch=joblib.load("../data_folder/deltaphish/preprocessed/phish_sub_test_x_ch_pa_u.pkl")
phish_sub_test_url_x_ch=pd.concat([phish_sub_test_x_ch.iloc[:,0:27],phish_sub_test_x_ch.iloc[:,49:58]],axis=1)
phish_sub_test_html_x_ch=phish_sub_test_x_ch.iloc[:,27:49]


phish_sub_test_x_ch_re=np.asarray(phish_sub_test_x_ch).reshape(phish_sub_test_x_ch.shape[0],phish_sub_test_x_ch.shape[1],1)
phish_sub_test_url_x_ch_re=np.asarray(phish_sub_test_url_x_ch).reshape(phish_sub_test_url_x_ch.shape[0],phish_sub_test_url_x_ch.shape[1],1)
phish_sub_test_html_x_ch_re=np.asarray(phish_sub_test_html_x_ch).reshape(phish_sub_test_html_x_ch.shape[0],phish_sub_test_html_x_ch.shape[1],1)

print('cnn_recall_combined_pa_u:')
base_sub_cnn_recall_com=get_base_recall_cnn(model_cnn_combined,phish_sub_test_x_ch_re,phish_sub_test_y)
print('cnn_recall_url_pa_u:')
base_sub_cnn_recall_url=get_base_recall_cnn(model_cnn_url,phish_sub_test_url_x_ch_re,phish_sub_test_y)
print('cnn_recall_html_pa_u:')
base_sub_cnn_recall_html=get_base_recall_cnn(model_cnn_html,phish_sub_test_html_x_ch_re,phish_sub_test_y)



cnn_recall_combined_pa_u:
cnn_recall 0.98
cnn_recall_url_pa_u:
cnn_recall 0.62
cnn_recall_html_pa_u:
cnn_recall 0.46


# pa_r attack- refer to Fig 5b, bottom row, blue bars in middle group of bars

In [14]:
phish_sub_test_x_ch=joblib.load("../data_folder/deltaphish/preprocessed/phish_sub_test_x_ch_pa_r.pkl")

phish_sub_test_url_x_ch=pd.concat([phish_sub_test_x_ch.iloc[:,0:27],phish_sub_test_x_ch.iloc[:,49:58]],axis=1)
phish_sub_test_html_x_ch=phish_sub_test_x_ch.iloc[:,27:49]

phish_sub_test_x_ch_re=np.asarray(phish_sub_test_x_ch).reshape(phish_sub_test_x_ch.shape[0],phish_sub_test_x_ch.shape[1],1)
phish_sub_test_url_x_ch_re=np.asarray(phish_sub_test_url_x_ch).reshape(phish_sub_test_url_x_ch.shape[0],phish_sub_test_url_x_ch.shape[1],1)
phish_sub_test_html_x_ch_re=np.asarray(phish_sub_test_html_x_ch).reshape(phish_sub_test_html_x_ch.shape[0],phish_sub_test_html_x_ch.shape[1],1)

print('cnn_recall_combined_pa_r:')
base_sub_cnn_recall_com=get_base_recall_cnn(model_cnn_combined,phish_sub_test_x_ch_re,phish_sub_test_y)
print('cnn_recall_url_pa_r:')
base_sub_cnn_recall_url=get_base_recall_cnn(model_cnn_url,phish_sub_test_url_x_ch_re,phish_sub_test_y)
print('cnn_recall_html_pa_r:')
base_sub_cnn_recall_html=get_base_recall_cnn(model_cnn_html,phish_sub_test_html_x_ch_re,phish_sub_test_y)




cnn_recall_combined_pa_r:
cnn_recall 0.57
cnn_recall_url_pa_r:
cnn_recall 0.63
cnn_recall_html_pa_r:
cnn_recall 0.71


# pa_c attack- refer to Fig 5b, bottom row, red bars in middle group of bars

In [15]:
phish_sub_test_x_ch=joblib.load("../data_folder/deltaphish/preprocessed/phish_sub_test_x_ch_pa_c.pkl")
phish_sub_test_url_x_ch=pd.concat([phish_sub_test_x_ch.iloc[:,0:27],phish_sub_test_x_ch.iloc[:,49:58]],axis=1)
phish_sub_test_html_x_ch=phish_sub_test_x_ch.iloc[:,27:49]

phish_sub_test_x_ch_re=np.asarray(phish_sub_test_x_ch).reshape(phish_sub_test_x_ch.shape[0],phish_sub_test_x_ch.shape[1],1)
phish_sub_test_url_x_ch_re=np.asarray(phish_sub_test_url_x_ch).reshape(phish_sub_test_url_x_ch.shape[0],phish_sub_test_url_x_ch.shape[1],1)
phish_sub_test_html_x_ch_re=np.asarray(phish_sub_test_html_x_ch).reshape(phish_sub_test_html_x_ch.shape[0],phish_sub_test_html_x_ch.shape[1],1)

print('cnn_recall_combined_pa_c:')
base_sub_cnn_recall_com=get_base_recall_cnn(model_cnn_combined,phish_sub_test_x_ch_re,phish_sub_test_y)
print('cnn_recall_url_pa_c:')
base_sub_cnn_recall_url=get_base_recall_cnn(model_cnn_url,phish_sub_test_url_x_ch_re,phish_sub_test_y)
print('cnn_recall_html_pa_c:')
base_sub_cnn_recall_html=get_base_recall_cnn(model_cnn_html,phish_sub_test_html_x_ch_re,phish_sub_test_y)



cnn_recall_combined_pa_c:
cnn_recall 0.99
cnn_recall_url_pa_c:
cnn_recall 0.62
cnn_recall_html_pa_c:
cnn_recall 0.46


# ma_u attack- refer to Fig 5b, bottom row, green bars  of the rightmost group of bars

In [16]:
phish_sub_test_x_ch=phish_sub_test_x.copy()
phish_sub_test_x_ch['URL_length']=-1
phish_sub_test_x_ch['URL_IP']=-1
phish_sub_test_x_ch['URL_redirect']=-1
phish_sub_test_x_ch['URL_shortener']=1
phish_sub_test_x_ch['URL_subdomains']=-1
phish_sub_test_x_ch['URL_at']=-1 
phish_sub_test_x_ch['URL_dash']=-1  
phish_sub_test_x_ch['URL_numberofCommonTerms']=-1 
phish_sub_test_x_ch['URL_checkNumerical']=-1 
phish_sub_test_x_ch['URL_checkPathExtend']=-1
phish_sub_test_x_ch['URL_checkPunycode']=-1
phish_sub_test_x_ch['URL_checkSensitiveWord']=0
phish_sub_test_x_ch['URL_checkTLDinSub']=-1
phish_sub_test_x_ch['URL_checkTLDinPath']=-1
phish_sub_test_x_ch['URL_totalWordUrl']=0

phish_sub_test_x_ch['URL_longestWordUrl']=0
phish_sub_test_x_ch['URL_longestWordHost']=0
phish_sub_test_x_ch['URL_longestWordPath']=0
phish_sub_test_x_ch['URL_averageWordUrl']=1000
phish_sub_test_x_ch['URL_averageWordHost']=0
phish_sub_test_x_ch['URL_averageWordPath']=1000
phish_sub_test_x_ch['REP_checkGI']=-1

phish_sub_test_url_x_ch=pd.concat([phish_sub_test_x_ch.iloc[:,0:27],phish_sub_test_x_ch.iloc[:,49:58]],axis=1)
phish_sub_test_html_x_ch=phish_sub_test_x_ch.iloc[:,27:49]

phish_sub_test_x_ch_re=np.asarray(phish_sub_test_x_ch).reshape(phish_sub_test_x_ch.shape[0],phish_sub_test_x_ch.shape[1],1)
phish_sub_test_url_x_ch_re=np.asarray(phish_sub_test_url_x_ch).reshape(phish_sub_test_url_x_ch.shape[0],phish_sub_test_url_x_ch.shape[1],1)
phish_sub_test_html_x_ch_re=np.asarray(phish_sub_test_html_x_ch).reshape(phish_sub_test_html_x_ch.shape[0],phish_sub_test_html_x_ch.shape[1],1)

print('cnn_recall_combined_ma_u:')
base_sub_cnn_recall_com=get_base_recall_cnn(model_cnn_combined,phish_sub_test_x_ch_re,phish_sub_test_y)
print('cnn_recall_url_ma_u:')
base_sub_cnn_recall_url=get_base_recall_cnn(model_cnn_url,phish_sub_test_url_x_ch_re,phish_sub_test_y)
print('cnn_recall_html_ma_u:')
base_sub_cnn_recall_html=get_base_recall_cnn(model_cnn_html,phish_sub_test_html_x_ch_re,phish_sub_test_y)



cnn_recall_combined_ma_u:
cnn_recall 0.00
cnn_recall_url_ma_u:
cnn_recall 0.00
cnn_recall_html_ma_u:
cnn_recall 0.82


# ma_r attack- refer to Fig 5b, bottom row, blue bars  of the rightmost group of bars

In [17]:
phish_sub_test_x_ch=phish_sub_test_x.copy()

phish_sub_test_x_ch['HTML_Objects']=-1
phish_sub_test_x_ch['HTML_metaScripts']=-1
phish_sub_test_x_ch['HTML_FrequentDomain']=-1
phish_sub_test_x_ch['HTML_Commonpage']=1#0#1
phish_sub_test_x_ch['HTML_CommonPageRatioinFooter']=1#1
phish_sub_test_x_ch['HTML_popUp']=-1
phish_sub_test_x_ch['HTML_RightClick']=-1
phish_sub_test_x_ch['HTML_DomainwithCopyright']=-1
phish_sub_test_x_ch['HTML_HiddenInfo_div']=-1
phish_sub_test_x_ch['HTML_nullLinksinWeb']=1
phish_sub_test_x_ch['HTML_nullLinksinFooter']=0#1
phish_sub_test_x_ch['HTML_BrokenLink']=1
phish_sub_test_x_ch['HTML_LoginForm']=-1
phish_sub_test_x_ch['HTML_HiddenInfo_input']=-1
phish_sub_test_x_ch['HTML_TitleUrlBrand']=-1
phish_sub_test_x_ch['HTML_favicon']=-1
phish_sub_test_x_ch['HTML_statusBarMod']=-1
phish_sub_test_x_ch['HTML_css']=-1
phish_sub_test_x_ch['HTML_anchors']=-1

phish_sub_test_url_x_ch=pd.concat([phish_sub_test_x_ch.iloc[:,0:27],phish_sub_test_x_ch.iloc[:,49:58]],axis=1)
phish_sub_test_html_x_ch=phish_sub_test_x_ch.iloc[:,27:49]

phish_sub_test_x_ch_re=np.asarray(phish_sub_test_x_ch).reshape(phish_sub_test_x_ch.shape[0],phish_sub_test_x_ch.shape[1],1)
phish_sub_test_url_x_ch_re=np.asarray(phish_sub_test_url_x_ch).reshape(phish_sub_test_url_x_ch.shape[0],phish_sub_test_url_x_ch.shape[1],1)
phish_sub_test_html_x_ch_re=np.asarray(phish_sub_test_html_x_ch).reshape(phish_sub_test_html_x_ch.shape[0],phish_sub_test_html_x_ch.shape[1],1)

print('cnn_recall_combined_ma_r:')
base_sub_cnn_recall_com=get_base_recall_cnn(model_cnn_combined,phish_sub_test_x_ch_re,phish_sub_test_y)
print('cnn_recall_url_ma_r:')
base_sub_cnn_recall_url=get_base_recall_cnn(model_cnn_url,phish_sub_test_url_x_ch_re,phish_sub_test_y)
print('cnn_recall_html_ma_r:')
base_sub_cnn_recall_html=get_base_recall_cnn(model_cnn_html,phish_sub_test_html_x_ch_re,phish_sub_test_y)


cnn_recall_combined_ma_r:
cnn_recall 0.00
cnn_recall_url_ma_r:
cnn_recall 0.63
cnn_recall_html_ma_r:
cnn_recall 0.00


# ma_c attack- refer to Fig 5b, bottom row, red bars  of the rightmost group of bars

In [18]:
phish_sub_test_x_ch=phish_sub_test_x.copy()
phish_sub_test_x_ch['URL_length']=-1
phish_sub_test_x_ch['URL_IP']=-1#single no change
phish_sub_test_x_ch['URL_redirect']=-1#
phish_sub_test_x_ch['URL_shortener']=1 #single no change
phish_sub_test_x_ch['URL_subdomains']=-1
phish_sub_test_x_ch['URL_at']=-1 
phish_sub_test_x_ch['URL_dash']=-1  
phish_sub_test_x_ch['URL_numberofCommonTerms']=-1 
phish_sub_test_x_ch['URL_checkNumerical']=-1 #single no change
phish_sub_test_x_ch['URL_checkPathExtend']=-1#single no change
phish_sub_test_x_ch['URL_checkPunycode']=-1#single no change
phish_sub_test_x_ch['URL_checkSensitiveWord']=0
phish_sub_test_x_ch['URL_checkTLDinSub']=-1
phish_sub_test_x_ch['URL_checkTLDinPath']=-1
phish_sub_test_x_ch['URL_totalWordUrl']=0

phish_sub_test_x_ch['URL_longestWordUrl']=0
phish_sub_test_x_ch['URL_longestWordHost']=0 
phish_sub_test_x_ch['URL_longestWordPath']=0
phish_sub_test_x_ch['URL_averageWordUrl']=1000
phish_sub_test_x_ch['URL_averageWordHost']=0
phish_sub_test_x_ch['URL_averageWordPath']=1000
 
phish_sub_test_x_ch['HTML_Objects']=-1
phish_sub_test_x_ch['HTML_metaScripts']=-1
phish_sub_test_x_ch['HTML_FrequentDomain']=-1
phish_sub_test_x_ch['HTML_Commonpage']=1#0#1
phish_sub_test_x_ch['HTML_CommonPageRatioinFooter']=1#1
 
phish_sub_test_x_ch['HTML_popUp']=-1
phish_sub_test_x_ch['HTML_RightClick']=-1
phish_sub_test_x_ch['HTML_DomainwithCopyright']=-1
phish_sub_test_x_ch['HTML_HiddenInfo_div']=-1
phish_sub_test_x_ch['HTML_nullLinksinWeb']=1#1#0
phish_sub_test_x_ch['HTML_nullLinksinFooter']=0#1
phish_sub_test_x_ch['HTML_BrokenLink']=1
phish_sub_test_x_ch['HTML_LoginForm']=-1
phish_sub_test_x_ch['HTML_HiddenInfo_input']=-1
phish_sub_test_x_ch['HTML_TitleUrlBrand']=-1
phish_sub_test_x_ch['HTML_favicon']=-1
phish_sub_test_x_ch['HTML_statusBarMod']=-1
phish_sub_test_x_ch['HTML_css']=-1
phish_sub_test_x_ch['HTML_anchors']=-1
phish_sub_test_x_ch['REP_checkGI']=-1

phish_sub_test_url_x_ch=pd.concat([phish_sub_test_x_ch.iloc[:,0:27],phish_sub_test_x_ch.iloc[:,49:58]],axis=1)
phish_sub_test_html_x_ch=phish_sub_test_x_ch.iloc[:,27:49]

phish_sub_test_x_ch_re=np.asarray(phish_sub_test_x_ch).reshape(phish_sub_test_x_ch.shape[0],phish_sub_test_x_ch.shape[1],1)
phish_sub_test_url_x_ch_re=np.asarray(phish_sub_test_url_x_ch).reshape(phish_sub_test_url_x_ch.shape[0],phish_sub_test_url_x_ch.shape[1],1)
phish_sub_test_html_x_ch_re=np.asarray(phish_sub_test_html_x_ch).reshape(phish_sub_test_html_x_ch.shape[0],phish_sub_test_html_x_ch.shape[1],1)

print('cnn_recall_combined_ma_u:')
base_sub_cnn_recall_com=get_base_recall_cnn(model_cnn_combined,phish_sub_test_x_ch_re,phish_sub_test_y)
print('cnn_recall_url_ma_u:')
base_sub_cnn_recall_url=get_base_recall_cnn(model_cnn_url,phish_sub_test_url_x_ch_re,phish_sub_test_y)
print('cnn_recall_html_ma_u:')
base_sub_cnn_recall_html=get_base_recall_cnn(model_cnn_html,phish_sub_test_html_x_ch_re,phish_sub_test_y)


cnn_recall_combined_ma_u:
cnn_recall 0.00
cnn_recall_url_ma_u:
cnn_recall 0.00
cnn_recall_html_ma_u:
cnn_recall 0.00
